## Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml  #using openml to import data
from sklearn.metrics import plot_roc_curve
from sklearn.model_selection import train_test_split, GridSearchCV      
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier  # added classification model
from sklearn.gaussian_process import GaussianProcessClassifier  # added classification model
from sklearn.gaussian_process.kernels import RBF  # added classification model
from sklearn.svm import LinearSVC  # added classification model
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer #transform different types

## Load titanic data

In [ ]:
X_initial, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
combine_dataset = pd.concat([X_initial, y], axis=1)
combine_dataset.head()

In [ ]:
# Save the dataset for R
combine_dataset.to_csv('./data/titanic_openml.csv', index=None)

## Part 1. Extend results with more variables

### Add the variable that was in created in the previous analysis (Practice 7) - family size

In [ ]:
combine_dataset['family size'] = combine_dataset['sibsp'] + combine_dataset['parch'] + 1

### Pipelines: Pre-Processing Stage

In [ ]:
features = ['age', 'fare', 'embarked', 'sex', 'pclass', 'family size']
X = combine_dataset[features].copy()

In [ ]:
numerical_features = ['age', 'fare', 'family size']  # family size added as new numerical feature/variable

# Applying SimpleImputer and StandardScaler into a pipeline
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']

# Applying SimpleImputer and then OneHotEncoder into another pipeline
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

data_transformer = ColumnTransformer(
    transformers=[
        ('numerical', numerical_transformer, numerical_features),
        ('categorical', categorical_transformer, categorical_features)]) 

### Create train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=1)

### Set parameters

In [ ]:
param_grid = {
    'data_transformer__numerical__imputer__strategy': ['mean', 'median'],
    'data_transformer__categorical__imputer__strategy': ['constant','most_frequent']
}

### Extend the results: logistic regression

In [ ]:
pipe_lr = Pipeline(steps=[('data_transformer', data_transformer),
                          ('pipe_lr', LogisticRegression(max_iter=10000, penalty='none'))]) #penalty='l2' is default

grid_lr = GridSearchCV(pipe_lr, param_grid=param_grid)
grid_lr.fit(X_train, y_train);

# Reference -- https://www.statisticshowto.com/regularization/
# L1 regularization adds an L1 penalty equal to the absolute value of the magnitude of coefficients. Lasso regression uses this method.
# L2 regularization adds an L2 penalty equal to the square of the magnitude of coefficients. Ridge regression uses this method.

### Extend the results: gradient boosting

In [ ]:
pipe_gdb = Pipeline(steps=[('data_transformer', data_transformer),
       ('pipe_gdb',GradientBoostingClassifier(random_state=2))])

grid_gdb = GridSearchCV(pipe_gdb, param_grid=param_grid)
grid_gdb.fit(X_train, y_train);

## 2. Extend the results with other classification methods

The following classification methods that are not demonstrated in the lecture:

* Penalised logistic regression
* Classification trees
* Random forests
* Gaussian process classification
* Support vector machines 

In [ ]:
# Penalised logistic regression
pipe_plr = Pipeline(steps=[('data_transformer', data_transformer),
                           ('pipe_plr', LogisticRegression(penalty='l1', max_iter=10000, tol=0.01, solver='saga'))])
grid_plr = GridSearchCV(pipe_plr, param_grid=param_grid)
grid_plr.fit(X_train, y_train);

# Reference -- https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# tol=0.01 -- tolerance for stopping criteria; stop searching for a minimum (or maximum) once some tolerance is achieved
# solver='saga' -- algorithm to use in the optimization problem; 'sag' and 'saga' are faster for large datasets

In [ ]:
# Classification tree
pipe_tree = Pipeline(steps=[('data_transformer', data_transformer),
                           ('pipe_tree', DecisionTreeClassifier(random_state=0))])
grid_tree = GridSearchCV(pipe_tree, param_grid=param_grid)
grid_tree.fit(X_train, y_train);

In [ ]:
# Random forest
pipe_rf = Pipeline(steps=[('data_transformer', data_transformer),
                           ('pipe_rf', RandomForestClassifier(random_state=0))])
grid_rf = GridSearchCV(pipe_rf, param_grid=param_grid)
grid_rf.fit(X_train, y_train);

In [ ]:
# Gaussian process classification
kernel = 1.0 * RBF(1.0)
pipe_gp = Pipeline(steps=[('data_transformer', data_transformer),
                          ('pipe_gp',  GaussianProcessClassifier(kernel=kernel, random_state=0))])
grid_gp = GridSearchCV(pipe_gp, param_grid=param_grid)
grid_gp.fit(X_train, y_train);

# References:
# https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessClassifier.html
# https://machinelearningmastery.com/gaussian-processes-for-classification-with-python/

In [ ]:
# Support vector machines
pipe_svm = Pipeline(steps=[('data_transformer', data_transformer),
                           ('pipe_svm',  LinearSVC(random_state=0, max_iter=10000, tol=0.01))])
grid_svm = GridSearchCV(pipe_svm, param_grid=param_grid)
grid_svm.fit(X_train, y_train);

In [ ]:
# Support vector machines (with additional hyperparameter tuning)
# Note: LinearSVC generates a linear classifier, while SVC lets you choose non-linear kernels

pipe_svc = Pipeline(steps=[('data_transformer', data_transformer),
                           ('pipe_svc',  SVC())])

# Hyperparameter tuning
param_grid2 = param_grid.copy()  #copy and extend param_grid
param_grid2['pipe_svc__kernel']=['rbf', 'poly','sigmoid']
param_grid2['pipe_svc__C']=[0.1, 1, 10]

#Alternatively, you can use '**' to pass multiple arguments to a function directly using a dictionary
#param_grid2 = { **param_grid,
#               'pipe_svc__kernel':['rbf', 'poly','sigmoid'],
#               'pipe_svc__C':[0.1, 1, 10]
#              }
                        
grid_svc = GridSearchCV(pipe_svc, param_grid=param_grid2)
grid_svc.fit(X_train, y_train);

grid_svc.best_params_
#grid_svc.best_estimator_
#grid_svc.best_score_

## Compare performance of classification models by the ROC curve

In [ ]:
ax, fig = plt.subplots(figsize=(8,8))
ax = plt.gca()

plot_roc_curve(grid_lr, X_test, y_test, ax=ax, name='Logistic Regression')
plot_roc_curve(grid_plr, X_test, y_test, ax=ax, name='Penalised logistic regression')
plot_roc_curve(grid_gdb, X_test, y_test, ax=ax, name='Gradient Boosting')
plot_roc_curve(grid_tree, X_test, y_test, ax=ax, name='Classification trees')
plot_roc_curve(grid_rf, X_test, y_test, ax=ax, name='Random forests')
plot_roc_curve(grid_gp, X_test, y_test, ax=ax, name='Gaussian process classification')
plot_roc_curve(grid_svm, X_test, y_test, ax=ax, name='Support vector machines (LinearSVC)')
plot_roc_curve(grid_svc, X_test, y_test, ax=ax, name='Support vector machines (SVC)')
plt.plot([0, 1], [0, 1], color='black', lw=1, linestyle='--')
plt.show()